In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Import base classifiers
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from baselines import AdaFairClassifier
from imbens.ensemble import SMOTEBoostClassifier, SMOTEBaggingClassifier, RUSBoostClassifier, UnderBaggingClassifier, SelfPacedEnsembleClassifier
from fairlearn.postprocessing import ThresholdOptimizer
from fairens import FairAugEnsemble, FairEnsemble

# Import utilities
from data import FairDataset    # This is a custom class that we will use to load the datasets
from eval import evaluate_multi_split, verbose_print
from trainer import Trainer
from utils import seed_generator, dict_info

pip install 'aif360[AdversarialDebiasing]'


In [3]:
import sklearn
sklearn.__version__

'1.3.2'

In [4]:
"""Load Datasets"""

dataset_kwargs = {
    'y_col': 'label',
    'train_size': 0.6,
    'val_size': 0.2,
    'test_size': 0.2,
    'concat_train_val': True,
    'normalize': True,
    'random_state': 42,
}

all_datasets = {
    'compas': ['sex', 'race'],
    'adult': ['gender', 'race'],
    'bank': ['age', 'marital=married'],
    # 'lsa_unfair_gender_race': ['gender', 'race'],
}

"""
Create a dictionary of datasets: dataset_zoo
key: dataset name
value: FairDataset object
"""
dataset_zoo = {}
for dataname, s_attrs in all_datasets.items():
    for s_attr in s_attrs:
        dataset = FairDataset(
            dataname=dataname,
            csv_path=f'./data/{dataname}.csv',
            s_col=s_attr,
            **dataset_kwargs
        )
        dataset_zoo[dataset.fullname] = dataset

        # dataset.describe()
        dataset.brief()

# Print the information of the datasets and models
print(
    f"////// Dataset ZOO //////\n"
    f"{dict_info(dataset_zoo)}\n"
)

dataset_zoo_subset = {
    'compas_sex': dataset_zoo['compas_sex'],
    'compas_race': dataset_zoo['compas_race'],
}

Dataset    : compas (5875, 12) load from ./data/compas.csv
Sens/Res   : sex/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 929, 1: 3771} | grp_pos_ratio: {0: 0.3617, 1: 0.4916}
test       | size {0: 232, 1: 943} | grp_pos_ratio: {0: 0.3621, 1: 0.491}

Dataset    : compas (5875, 12) load from ./data/compas.csv
Sens/Res   : race/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 1878, 1: 2822} | grp_pos_ratio: {0: 0.3946, 1: 0.5135}
test       | size {0: 469, 1: 706} | grp_pos_ratio: {0: 0.3945, 1: 0.5127}

Dataset    : adult (45222, 99) load from ./data/adult.csv
Sens/Res   : gender/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 11756, 1: 24421} | grp_pos_ratio: {0: 0.1136, 1: 0.3125}
test       | size {0: 2939, 1: 6106} | grp_pos_ratio: {0: 0.1136, 1: 0.3125}

Dataset    : adult (45222, 99) load from ./data/adult.csv
Sens/Res   : race/labe

In [5]:
from trainer import Benchmarker
from baselines import ReweightClassifier, ReductionClassifier

ensemble_kwargs = {
    'n_estimators': 5,
    'random_state': 42,
}
single_ensemble_kwargs = {
    'n_estimators': 1,
    'random_state': 42,
}

base_models = {
    'LR': LogisticRegression(),
    # 'KN': KNeighborsClassifier(),
    # 'DT': DecisionTreeClassifier(max_depth=None),
    # 'MLP': MLPClassifier(hidden_layer_sizes=(8), max_iter=50),
    # 'ADA': AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=None), n_estimators=5),
    # 'BAG': BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=None), n_estimators=5),
}

baselines = {
    # 'AdaBoost': (AdaBoostClassifier, {**ensemble_kwargs}),
    # 'Bagging': (BaggingClassifier, {**ensemble_kwargs}),
    # 'RUSBoost': (RUSBoostClassifier, {**ensemble_kwargs}),
    # 'UnderBag': (UnderBaggingClassifier, {**ensemble_kwargs}),
    # 'SMBoost': (SMOTEBoostClassifier, {**ensemble_kwargs}),
    # 'SMBag': (SMOTEBaggingClassifier, {**ensemble_kwargs}),
    'Reweight': (ReweightClassifier, {}),
    'ReductionDP': (ReductionClassifier, {'constraints': 'DemographicParity'}),
    'ReductionEO': (ReductionClassifier, {'constraints': 'EqualizedOdds'}),
    'ThresDP': (ThresholdOptimizer, {'constraints': 'demographic_parity'}),
    'ThresEO': (ThresholdOptimizer, {'constraints': 'equalized_odds'}),
    'AdaFair': (AdaFairClassifier, {'saIndex': 0, 'saValue': 0, 'CSB': 'CSB2', **ensemble_kwargs}),
}

benchmark = Benchmarker(
    base_models=base_models,
    baselines=baselines,
    datasets=dataset_zoo_subset,
    random_state=42,
    dummy_strategy='stratified',
)
benchmark.run(n_runs=5, group_by='dataset', exception='ignore')

Initializing Benchmarker with:
Random seed: 42
Base models: ['LR']
Techniques:  ['Reweight', 'ReductionDP', 'ReductionEO', 'ThresDP', 'ThresEO', 'AdaFair']
Datasets:    ['compas_sex', 'compas_race']
# models:    7
# datasets:  2

Running All models ...
========== Start Running on Dataset: compas_sex ==========


Data: compas_sex | Model: Dummy          : 100%|██████████| 5/5 [00:00<00:00,  8.94it/s, ACC 0.497±0.021 | BACC 0.495±0.022 | DP 0.010±0.006 | EO 0.044±0.043 | SI 0.000±0.000 | AdvG {0: 4, 1: 1}]
Data: compas_sex | Model: LR             : 100%|██████████| 5/5 [00:00<00:00,  8.74it/s, ACC 0.696±0.006 | BACC 0.690±0.006 | DP 0.336±0.016 | EO 0.382±0.039 | SI 0.210±0.013 | AdvG {1: 5}]
Data: compas_sex | Model: LR_Reweight    : 100%|██████████| 5/5 [00:01<00:00,  2.71it/s, ACC 0.689±0.009 | BACC 0.677±0.009 | DP 0.275±0.022 | EO 0.343±0.055 | SI 0.166±0.010 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ReductionDP : 100%|██████████| 5/5 [00:01<00:00,  4.09it/s, ACC 0.689±0.010 | BACC 0.686±0.010 | DP 0.105±0.041 | EO 0.086±0.043 | SI 0.000±0.000 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ReductionEO : 100%|██████████| 5/5 [00:01<00:00,  3.91it/s, ACC 0.687±0.009 | BACC 0.682±0.009 | DP 0.128±0.028 | EO 0.109±0.033 | SI 0.000±0.000 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ThresDP     : 1

========== Run Time on Dataset compas_sex: 0m8s ==========
Results on Data: compas_sex
Model: Dummy          | ACC 0.497±0.021            | BACC 0.495±0.022            | DP 0.010±0.006            | EO 0.044±0.043            | SI 0.000±0.000           
Model: LR             | ACC 0.696±0.006            | BACC 0.690±0.006            | DP 0.336±0.016            | EO 0.382±0.039            | SI 0.210±0.013           
Model: LR_Reweight    | ACC 0.689±0.009 (-3.59%)   | BACC 0.677±0.009 (-6.74%)   | DP 0.275±0.022 (-18.18%)  | EO 0.343±0.055 (-10.36%)  | SI 0.166±0.010 (-20.75%)  | FURG 11.26    | FUTR 3.18    
Model: LR_ReductionDP | ACC 0.689±0.010 (-3.42%)   | BACC 0.686±0.010 (-2.08%)   | DP 0.105±0.041 (-68.82%)  | EO 0.086±0.043 (-77.60%)  | SI 0.000±0.000 (-100.00%) | FURG 79.39    | FUTR 29.87   
Model: LR_ReductionEO | ACC 0.687±0.009 (-4.36%)   | BACC 0.682±0.009 (-3.93%)   | DP 0.128±0.028 (-61.80%)  | EO 0.109±0.033 (-71.38%)  | SI 0.000±0.000 (-100.00%) | FURG 73.58    | FUTR 1

Data: compas_race | Model: Dummy          : 100%|██████████| 5/5 [00:00<00:00,  9.62it/s, ACC 0.491±0.015 | BACC 0.489±0.016 | DP 0.014±0.012 | EO 0.035±0.023 | SI 0.000±0.000 | AdvG {1: 3, 0: 2}]
Data: compas_race | Model: LR             : 100%|██████████| 5/5 [00:00<00:00,  8.94it/s, ACC 0.685±0.011 | BACC 0.680±0.011 | DP 0.310±0.028 | EO 0.337±0.036 | SI 0.058±0.013 | AdvG {1: 5}]
Data: compas_race | Model: LR_Reweight    : 100%|██████████| 5/5 [00:01<00:00,  2.76it/s, ACC 0.677±0.012 | BACC 0.667±0.012 | DP 0.275±0.028 | EO 0.319±0.040 | SI 0.070±0.008 | AdvG {1: 5}]
Data: compas_race | Model: LR_ReductionDP : 100%|██████████| 5/5 [00:01<00:00,  4.12it/s, ACC 0.643±0.035 | BACC 0.630±0.036 | DP 0.126±0.052 | EO 0.168±0.049 | SI 0.000±0.000 | AdvG {1: 5}]
Data: compas_race | Model: LR_ReductionEO : 100%|██████████| 5/5 [00:01<00:00,  3.94it/s, ACC 0.652±0.039 | BACC 0.642±0.039 | DP 0.132±0.063 | EO 0.158±0.073 | SI 0.000±0.000 | AdvG {1: 5}]
Data: compas_race | Model: LR_ThresDP  